In [129]:
pip install requests pandas openpyxl



[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [130]:
import pandas as pd
import sqlite3
import requests
import csv
import os


In [131]:
#Configuración de API
api_key = '10c299b6d3f1cff2d270c57cf73e11fe'
URL = f'https://gnews.io/api/v4/search?q=ciudad+de+mexico&lang=es&country=mx&max=90&apikey={api_key}'

In [132]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect('seguimiento.db')
cursor = conn.cursor()

In [133]:
#Crear tabla
cursor.execute('''CREATE TABLE IF NOT EXISTS seguimiento (
               id INTEGER PRIMARY KEY AUTOINCREMENT,
               fecha TEXT,
               encabezado TEXT UNIQUE,
               description TEXT,
               fuente TEXT,
               url TEXT UNIQUE)''')

conn.commit()

In [134]:
#Obtener noticias
respuesta = requests.get(URL)
data = respuesta.json()

In [135]:
for articulo in data.get('articles', []):
    try:
        cursor.execute('''
        INSERT OR REPLACE INTO seguimiento(fecha, encabezado, description, fuente, url)
        VALUES (?, ?, ?, ?, ?)''',
        (articulo['publishedAt'], articulo['title'], articulo['description'], articulo['source']['name'], articulo['url']))
        
        # Agregar a la lista de noticias para el CSV
        noticias.append((articulo['publishedAt'], articulo['title'], articulo['description'], articulo['source']['name'], articulo['url']))
    except sqlite3.Error as e:
        print(f'Error al insertar la noticia: {articulo["title"]}, {e}')


In [136]:
#Guardar cambios y cerrar conexión
conn.commit()
conn.close()

In [137]:
# Guardar los resultados en un archivo Excel (.xlsx) con pandas
df = pd.DataFrame(noticias)

# Guardar el archivo en la ubicación actual de trabajo
df.to_excel('seguimiento_noticias.xlsx', index=False, engine='openpyxl')

print('El archivo Excel se guardó en:', os.getcwd())

El archivo Excel se guardó en: /Users/bch041097/VSCode


In [138]:
# Guardar los resultados en un archivo CSV
with open('seguimiento_noticias.csv', 'w', newline='', encoding='UTF-8') as f:
    writer = csv.writer(f)
    
    # Escribir encabezados (puedes usar el nombre de las columnas en lugar de esto si quieres)
    writer.writerow(['id', 'fecha', 'encabezado', 'descripcion', 'fuente', 'url'])
    
    # Escribir los datos
    writer.writerows(noticias)


In [139]:
print('el archivo csv se guardó en:', os.getcwd())

el archivo csv se guardó en: /Users/bch041097/VSCode
